In [1]:
from pathlib import Path
from eo_tools.S1.core import read_metadata
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from eo_tools.S1.core import S1IWSwath

In [2]:
dir_slc = '/data/S1/S1A_IW_SLC__1SDV_20240410T180524_20240410T180551_053375_067920_A0F7/S1A_IW_SLC__1SDV_20240410T180524_20240410T180551_053375_067920_A0F7.SAFE'
#dir_out = '/data/res/test-slc-proc/S1_2024-04-10-180524'
safe_dir = dir_slc
iw = 1
pol = "vv"

In [3]:
# read calibration file
dir_cal = Path(safe_dir) / "annotation" / "calibration"
pth_cal = dir_cal.glob(f"calibration*iw{iw}*{pol}*.xml")
try:
    pth_cal = list(pth_cal)[0]
except IndexError:
    raise FileNotFoundError("Calibration file is missing.")

In [6]:
cal_meta = read_metadata(pth_cal)['calibration']
# parse calibration grids
cal_list = cal_meta["calibrationVectorList"]["calibrationVector"]
str_cols = cal_list[0]["pixel"]["#text"]
cols = np.array(list(map(int, str_cols.split(" "))), dtype=int)
grid_sigma = np.zeros((len(cal_list), len(cols)), dtype = "float64")
grid_gamma= np.zeros((len(cal_list), len(cols)), dtype = "float64")
list_lines = []
for i, it in enumerate(cal_list):
    list_lines.append(int(it["line"]))
    str_sigma = it["sigmaNought"]["#text"]
    line_sigma = list(map(float, str_sigma.split(" ")))
    grid_sigma[i] = line_sigma

    str_gamma = it["gamma"]["#text"]
    line_gamma = list(map(float, str_gamma.split(" ")))
    grid_gamma[i] = line_gamma
rows = np.array(list_lines, dtype=int)
swath = S1IWSwath(dir_slc, iw=1, pol="vv") 
naz = swath.lines_per_burst
nrg = swath.samples_per_burst
grid_arr_rg, grid_arr_az = np.meshgrid(np.arange(nrg), np.arange(naz))
grid_cal_rg, grid_cal_az = np.meshgrid(cols, rows)
interp = RegularGridInterpolator((rows, cols), grid_gamma, method="linear")
img = interp((grid_arr_az, grid_arr_rg))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(grid_arr_az, interpolation="none")
# plt.colorbar(fraction=0.046, pad=0.04)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)